In [59]:
pip install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.8 MB 7.0 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1634001 sha256=e41d602d65ce7343ab18ec34a6cbcb9004fa9586625c17958c13bf9f4c6c25ab
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [94]:
import pandas as pd
from surprise import KNNWithMeans, KNNBasic, KNNBaseline
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split

In [52]:
movies = pd.read_csv('movies.csv', sep='::', encoding="ISO-8859-1", names=['movieId', 'title', 'genres'])

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [64]:
ratings = pd.read_csv('ratings.csv', sep='::', names=['userId', 'movieId', 'rating', 'timestamp'])

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [80]:
movies_with_ratings = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

In [81]:
#movies_with_ratings[movies_with_ratings.userId == 2.0].title.unique()

In [68]:
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

In [69]:
dataset.head()

,uid,iid,rating
0,1.0,Toy Story (1995),5.0
1,6.0,Toy Story (1995),4.0
2,8.0,Toy Story (1995),4.0
3,9.0,Toy Story (1995),5.0
4,10.0,Toy Story (1995),5.0


In [70]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset, reader)

In [71]:
trainset, testset = train_test_split(data, test_size=.15, random_state=42)

In [82]:
algo = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': True})
algo.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [83]:
test_pred = algo.test(testset)

In [84]:
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8892


0.8892192259630961

In [85]:
algo = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': False})
algo.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [86]:
test_pred = algo.test(testset)

In [87]:
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8596


0.8595617942697337

In [89]:
from surprise import SVD

In [90]:
%%time
algo = SVD(n_factors=20, n_epochs=20)
algo.fit(trainset)

CPU times: user 29.7 s, sys: 84.8 ms, total: 29.8 s
Wall time: 30.7 s


In [91]:
test_pred = algo.test(testset)

In [92]:
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8717


0.8716676015215192

In [96]:
algo = KNNBaseline(k=50, sim_options={'name': 'pearson_baseline', 'user_based': True})
algo.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [97]:
test_pred = algo.test(testset)

In [98]:
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8695


0.8695042655465262